# Data Analysis of CIA World Factbook with SQL

The CIA factbook, also known as the [World Factbook]((https://www.cia.gov/the-world-factbook/), is an annual publication of the US Central Intelligence Agency. It provides basic intelligience by summarizing information about countries and regions worldwide. The factbook contains a mix of demographic and geographic data, among many more.

<img src="https://miro.medium.com/max/700/1*QPDFLifvVMnzrIrx80e_bw.png" width=800 height=200/>

Source: [Dani Hensley](https://danihensley.medium.com/transparent-intelligence-the-cia-world-factbook-21907c0f49b1)


## Project and Data Overview

In this project, we'll use SQL in Jupyter Notebook to analyze data from **SQLite factbook db** database, which is available to download [here](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db). The goal is to identify the leading and lagging territories of the world based on demographic indicators like population, mortality, and fertility.

The Factbook contains demographic information like the following:

- `population` - the globle population.
- `population_growth` - the annual population growth rate, as a percentage.
- `area` - the total land and water area.

## Connect to the Database

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

## Understand the Data

We begin by querying the database to get information about the table.

In [2]:
%%sql
SELECT *
FROM sqlite_master
WHERE type = 'table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


There are two tables in the database, **sqlite_sequence** and **facts**. The sqlite_sequence does not contain any columns, whereas facts table has information on *population*, *birth rate*, *migration rate* and many more. We need this table for the analysis.

To run SQL queries, we add `%%sql` on its own line to the start of our query. Therefore, to run our query that returns the first five rows of the `facts` table in the database will look like this:

In [3]:
%%sql
-- Examine the first 5 rows in the facts table
SELECT *
FROM facts
LIMIT 5;  

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Executing the query above, we see the first few rows of our `facts` table. Here are the description of the 11 columns we have:

- `id` - entry row number.
- `code` - the country's [internet code](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db) 
- `name` - the name of the country.
- `area` - the country's total area (both land and water).
- `area_land` - the country's land area in square kilometers.
- `area_water` - the country's waterarea in square kilometers.
- `population` - the country's population.
- `population_growth` - the country's population growth as a percentage.
- `birth_rate` - the country's birth rate, or the number of births per year per 1,000 people.
- `death_rate` - the country's death rate, or the number of death per year per 1,000 people.
- `migration_rate` - the difference between the number of persons entering and leaving the country during the year per 1,000 persons.

Let's count the total number of records in the facts table:

In [4]:
%%sql
-- Count the total number of records
SELECT COUNT(*) AS num_records,
COUNT(DISTINCT name) AS unique_countries
FROM facts;

 * sqlite:///factbook.db
Done.


num_records,unique_countries
261,261


261 records and countries seem unintuitive! We know there are only 195 countries in the world right now.

The answer lies in the way CIA categories its information. As of july 2011, the World Factbook comprises 267 entities divided into the following categories:

- **Independent countries**: The CIA defines these as people "politically organized into a sovereign state with a definite territory". There are 195 entities in this category.
- **Others**: Places set apart from the list of independent countries. Currently, there are two: Taiwan and the European Union.
- **Dependencies and Areas of Special Sovereignty**: Places affiliated with another country. There are 58 in total.
- **Miscellaneous**: Antarctica and places in dispute. There are six entities.
- **Other entities**: The world and the oceans. There are five oceans and the world (the World entry is intended as a summary of the other entries).

Additional information about this classification can be found in the [world factbook wikipedia page](https://en.wikipedia.org/wiki/The_World_Factbook#:~:text=Entities%20listed,-Main%20article%3A%20List&text=The%20CIA%20defines%20these%20as,category%2C%20there%20are%20195%20entities.&text=Places%20set%20apart%20from%20the,Taiwan%20and%20the%20European%20Union.).

We can check for these occurrences where the rows are `non-country` in our data.

In [5]:
%%sql
-- Check for instances of "non-country" data
SELECT *
FROM facts
Where name LIKE '%union'
    OR name LIKE '%ocean'
    OR name LIKE '%island'
    OR name = 'World';

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
197,ee,European Union,4324782,None,None,513949445,0.25,10.2,10.2,2.5
199,kt,Christmas Island,135,135,0,1530,1.11,None,None,None
203,nf,Norfolk Island,36,36,0,2210,0.01,None,None,None
208,ip,Clipperton Island,6,6,0,None,None,None,None,None
222,bv,Bouvet Island,49,49,0,None,None,None,None,None
244,bq,Navassa Island,5,5,0,None,None,None,None,None
248,wq,Wake Island,6,6,0,None,None,None,None,None
256,xq,Arctic Ocean,None,None,None,None,None,None,None,None
257,zh,Atlantic Ocean,None,None,None,None,None,None,None,None
258,xo,Indian Ocean,None,None,None,None,None,None,None,None


Since these occurrences are present, we should be careful to avoid their influence on the analysis process.

We can now proceed to analysis with *some* understanding of the data and potential pitfalls to look out for.

## Explore the Total Population

To understand how population values are distributed in the data, we can compute the population summary statistics: the `minimum`, `mean` and `maximum`.

In [6]:
%%sql
SELECT MIN(population) AS min_pop,
       CAST(AVG(population) AS INT) AS avg_pop,
       MAX(population) AS max_pop
FROM facts;

 * sqlite:///factbook.db
Done.


min_pop,avg_pop,max_pop
0,62094928,7256490011


Surprising to see some countries/entities have no inhabitants. Another entity has a population of over 7 billion people. This population seems too good to be true. It is best to investigate what these entities are:

In [7]:
%%sql
-- Investigate the unusual countries/entities
SELECT name, population
FROM facts
WHERE population = (SELECT MIN(population) FROM facts)
    OR population = (SELECT MAX(population) FROM facts);

 * sqlite:///factbook.db
Done.


name,population
Antarctica,0
World,7256490011


Similar to our earlier observations, the presence of entries like *World* and *Antarctica* explains the unusual population values. According to the CIA Factbook [page for Antarctica](https://www.cia.gov/the-world-factbook/countries/antarctica/), Antarctica has no indigenous inhabitants, only research staff occasionally present at its many research stations:
<img src="https://s3.amazonaws.com/dq-content/257/fb_antarctica.png" width=800 height=800/>

Another interesting thing to note is the world population value `7256490011`. A quick search of world population trends reveals that the CIA could have compiled the facts data between 2014 and 2015.
<img src="https://nbviewer.org/github/Lippins/CIA_factbook_data_analysis/blob/main/images/world_population.png" width=400 height=400/>
Source: [Worldometer](https://www.worldometers.info/world-population/world-population-by-year/)

With these outliers in mind, we can compute a more appropriate descriptive statistics.

In [8]:
%%sql
SELECT MIN(population) AS min_pop,
       CAST(AVG(population) AS INT) AS avg_pop,
       MAX(population) AS max_pop
FROM facts
WHERE population != 0
    AND name != 'World';

 * sqlite:///factbook.db
Done.


min_pop,avg_pop,max_pop
48,32377011,1367485388


The minimum population is 48, and the maximum is over 1.3 billion - the average population centers around 32 million inhabitants. We can now identify the highest and least populated countries or entities.

### i. Top 5 Countries with Highest Total Population

In [9]:
%%sql
SELECT name AS top_5_populated,
       population
FROM facts
WHERE population <> 0
    AND name <> 'World'
ORDER BY 2 DESC
LIMIT 5; 

 * sqlite:///factbook.db
Done.


top_5_populated,population
China,1367485388
India,1251695584
European Union,513949445
United States,321368864
Indonesia,255993674


The European Union is not a country; it is better to filter out the EU. As an extra step, we will calculate the porportion of the world's population contained in each country.

In [10]:
%%sql
SELECT name AS '5 Most Populated Countries',
       population,
       ROUND(CAST(population AS FLOAT) /
             CAST((SELECT MAX(population) FROM facts) AS FLOAT), 3)
             AS world_population
FROM facts
WHERE population <> 0
    AND name NOT IN ('World', 'European Union')
ORDER BY 2 DESC
LIMIT 5; 

 * sqlite:///factbook.db
Done.


5 Most Populated Countries,population,world_population
China,1367485388,0.188
India,1251695584,0.172
United States,321368864,0.044
Indonesia,255993674,0.035
Brazil,204259812,0.028


China is the world's most populated country, with about 1.37 billion people. India follows behind with about 1.26 billion people. 36% of the world's total population comes from China (18.8%) and India (17.2%) alone.

The United States (4.4% of the world total), Indonesia (3.5%), and Brazil (2.8%) follow far behind and constitute a combined 11% of the world's population.

Altogether the top 5 countries contribute 46% to the global population.

### ii. Top 5 Countries with Least Total Population

In [11]:
%%sql
SELECT name AS '5 Least Populated Countries',
       population
FROM facts
WHERE population <> 0 
    AND name <> 'Antartica'
ORDER BY 2
LIMIT 5; 

 * sqlite:///factbook.db
Done.


5 Least Populated Countries,population
Pitcairn Islands,48
Cocos (Keeling) Islands,596
Holy See (Vatican City),842
Niue,1190
Tokelau,1337


The least populated territories are the Pitcairn Islands (48 inhabitants), Cocos Island (596), Holy See (842), Niue (1190), and Tokelau (1337).

Aside from the Holy See, the universal government of the Catholic Church that operates from the Vatican City, the least populated countries are dependencies. Pitcairn Island is a dependency of the United Kingdom; Cocos Island is a dependency of Australia; Niue and Tokelau are dependencies of New Zealand. [Here](https://www.worldatlas.com/geography/dependencies-and-territories-of-the-world.html) is a list of world's dependencies at the moment.

## Explore Population Density

The population density is the concentration of individuals within a species in a specific geographic locale. Population density data can be used to quantify demographic information and to assess relationships with ecosystems, human health, and infrastructure.

Our data provides variables that we can use to estimate population density as follows:

$$
Population\ density = \frac{Total\ population}{Land\ area\ (km^2)}
$$

Before computing population densities for each country, let's count for territories where population density or the land area may be zero or null.

In [12]:
%%sql
SELECT COUNT(*)
FROM facts
WHERE population = 0
    OR population IS NULL
    OR area_land = 0
    OR area_land IS NULL;

 * sqlite:///factbook.db
Done.


COUNT(*)
29


Indeed, 29 records meet these criteria. We will exclude these records.

### i. Top 10 Countries with the Highest Population Density

In [13]:
%%sql
SELECT name, area_land, population,
       CAST(CAST(population AS FLOAT) / 
            CAST(area_land AS FLOAT) AS INT)
            AS population_density
FROM facts

-- Filter out countries with zero or missing "population/land area"
WHERE name NOT IN (SELECT name
                   FROM facts
                   WHERE population = 0
                       OR population IS NULL
                       OR area_land = 0
                       OR area_land IS NULL)
ORDER BY 4 DESC
LIMIT 10;  

 * sqlite:///factbook.db
Done.


name,area_land,population,population_density
Macau,28,592731,21168
Monaco,2,30535,15267
Singapore,687,5674472,8259
Hong Kong,1073,7141106,6655
Gaza Strip,360,1869055,5191
Gibraltar,6,29258,4876
Bahrain,760,1346613,1771
Maldives,298,393253,1319
Malta,316,413965,1310
Bermuda,54,70196,1299


The Chinese territory of Macau has the world's highest population density at 21,168 $km^2$. Three of the top five entries are territories rather than countries, with first-place Macau joined by fellow Chinese territory Hong Kong in 4th place and the Palestinian territory Gaza Strip in 5th.

Notably, all of these locations are tiny. The top five have a combined land area of less than 3,000 $km^2$. Moreover, most are also highly urbanized centers of wealth and finance, with a tendency toward cities and skyscrapers rather than rolling fields or vast wilderness areas ([see here](https://worldpopulationreview.com/country-rankings/countries-by-density)).

Perhaps we could "redefine" a densely populated country to address the possible bias above. In our case, we will define a densely populated territory as one with a lower land area and a higher number of inhabitants compared to other territories.

Precisely, for a densely populated country or territory:

$$
Total\ population\ >\ Average\ world\ population\ \\
AND\ Land\ area\ <\ World\ average\ land\ area
$$

We can use this new rule to identify the most densely populated countries:

In [14]:
%%sql
-- Identify countries with zero or missing "population/land_area"
WITH t1 AS (SELECT name
            FROM facts
            WHERE population = 0
               OR population IS NULL
               OR area_land = 0
               OR area_land IS NULL),

-- Calculate population density for countries without missing data
    t2 AS (SELECT name, area_land, population,
              CAST(CAST(population AS FLOAT) /
                   CAST(area_land AS FLOAT) AS INT)
                   AS population_density
           FROM facts
           WHERE name NOT IN t1)
    
-- Isolate countries with high population and low land area
SELECT *
FROM t2
WHERE area_land < (SELECT AVG(area_land) FROM t2)
    AND population > (SELECT AVG(population) FROM t2)
ORDER BY population_density DESC
LIMIT 10; 

 * sqlite:///factbook.db
Done.


name,area_land,population,population_density
Bangladesh,130170,168957745,1297
"Korea, South",96920,49115196,506
Japan,364485,126919659,348
Philippines,298170,100998376,338
Vietnam,310070,94348835,304
United Kingdom,241930,64088222,264
Germany,348672,80854408,231
Nepal,143351,31551305,220
Italy,294140,61855120,210
Uganda,197100,37101745,188


Bangladesh leads in terms of high population and low land area with a population density of 1,297 people per $km^2$. Bangladesh's population density is over two times the values recorded for 2nd-placed South Korea. Japan, Philippines, and Vietnam follow behine.

### ii. Top 10 Countries with the Lowest Population Density

In [15]:
%%sql
SELECT name, area_land, population,
       ROUND(CAST(population AS FLOAT) /
             CAST(area_land AS FLOAT), 3)
             AS population_density
FROM facts

-- Filter out countries with zero or missing "population/land area"
WHERE name NOT IN (SELECT name
                   FROM facts
                   WHERE population = 0
                     OR population IS NULL
                     OR area_land = 0
                     OR area_land IS NULL)
ORDER BY 4
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,area_land,population,population_density
Greenland,2166086,57733,0.027
Svalbard,62045,1872,0.03
Falkland Islands (Islas Malvinas),12173,3361,0.276
Pitcairn Islands,47,48,1.021
Mongolia,1553556,2992908,1.926
Western Sahara,266000,570866,2.146
Namibia,823290,2212307,2.687
Australia,7682300,22751014,2.961
Iceland,100250,331918,3.311
Mauritania,1030700,3596702,3.49


The world's least-densely populated territory is Greenland, with a rate of 0.027 people per square km. Most of Greenland is covered in ice. Svalbard's remote and bitterly cold territory, located north of Norway, comes second on this list (0.03 people per square km). Falkland Island, Pitcairn Island, and Mongolia follow behind.

An interesting pattern is that most of these territories are remote, with challenging climate conditions, which may explain the low number of inhabitants (source: [WorldAtlas](https://www.worldatlas.com/articles/the-10-least-densely-populated-places-in-the-world-2015.html)).

Alternatively, we can explore low population density in the following context:

$$
Total\ population\ <\ Average\ world\ population \\ AND\ Land\ area\ >\ World\ average\ land\ area
$$

In [16]:
%%sql
-- Identify countries with zero or missing population/land area
WITH t1 AS (SELECT name
            FROM facts
            WHERE population = 0
                OR population IS NULL
                OR area_land = 0
                OR area_land IS NULL),

-- Calculate population density for countries without missing data
    t2 AS (SELECT name, area_land, population,
              ROUND(CAST(population AS FLOAT) /
                    CAST(area_land AS FLOAT), 3)
                    AS population_density
           FROM facts
           WHERE name NOT IN t1)

-- Isolate countries with high population and low land area    
SELECT *
FROM t2
WHERE population < (SELECT AVG(population) FROM t2)
    AND area_land > (SELECT AVG(area_land) FROM t2)
ORDER BY population_density
LIMIT 10; 

 * sqlite:///factbook.db
Done.


name,area_land,population,population_density
Greenland,2166086,57733,0.027
Mongolia,1553556,2992908,1.926
Namibia,823290,2212307,2.687
Australia,7682300,22751014,2.961
Mauritania,1030700,3596702,3.49
Libya,1759540,6411776,3.644
Botswana,566730,2182719,3.851
Kazakhstan,2699700,18157122,6.726
Central African Republic,622984,5391539,8.654
Chad,1259200,11631456,9.237


Greenland remains the world's least-densely populated territory. With a land area of over 2 million sqaure kilometers and only about 58,000 inhabitants, Greenland looks like the number-one recommended spot for those who want to feel like they've left all of humanity behind.

Based on the new criteria, Mongolia, Namibia, Australia, and Mauritania occupy the 2nd to 5th places, respectively.

## Mortality and Fertility

Two metrics determine the change in the world population: the number of babies born and people dying. [In 2021](https://statisticstimes.com/demographics/world-death-and-birth-rate.php), the crude death rate for the world was 7.64 deaths per thousand population, and the crude birth rate for the world is 17.76 births per thousand. These figures indicates a ratio of about 2.3 births for every death.

Let's query our data for the same information:

In [17]:
%%sql
SELECT birth_rate, death_rate,
       ROUND((birth_rate / death_rate), 2) AS birth_to_death_ratio
FROM facts
WHERE name = 'World';

 * sqlite:///factbook.db
Done.


birth_rate,death_rate,birth_to_death_ratio
18.6,7.8,2.38


Our data reveals a similar pattern, with about 2.4 births for every death.

### i. Top 10 Countries with the Highest Birth Rates

In [18]:
%%sql
SELECT name, birth_rate
FROM facts
WHERE name <> 'World'
ORDER BY 2 DESC
LIMIT 10; 

 * sqlite:///factbook.db
Done.


name,birth_rate
Niger,45.45
Mali,44.99
Uganda,43.79
Zambia,42.13
Burkina Faso,42.03
Burundi,42.01
Malawi,41.56
Somalia,40.45
Angola,38.78
Mozambique,38.58


Niger has the highest birth rate in the world at 45.45 average annual births per 1,000 people per year. Mali (44.99), Uganda (43.79), Zambia (42.13), and Burkina Faso (42.03) follow closely behind. All countries in the top-10 list belong to Africa. Africa's dominant birth rate is further emphasized in the figure below:

<img src="https://www.geocurrents.info/wp-content/uploads/2016/02/TFR-2010-2015-World-Map.png" width=600 height=300/>

Source: [Geocurrents](https://www.geocurrents.info/population-geography/total-fertility-rates-by-country-1950-and-2015)

### ii. Top 10 Countries with the Lowest Birth Rates

In [19]:
%%sql
SELECT name, birth_rate
FROM facts
WHERE name <> 'World'
-- Filter out entries that have missing birth rates
    AND birth_rate IS NOT NULL
ORDER BY 2
LIMIT 10; 

 * sqlite:///factbook.db
Done.


name,birth_rate
Monaco,6.65
Saint Pierre and Miquelon,7.42
Japan,7.93
Andorra,8.13
"Korea, South",8.19
Singapore,8.27
Slovenia,8.42
Germany,8.47
Taiwan,8.47
San Marino,8.63


On the other side of the scale, the CIA estimates Monaco has the lowest birth rate in the world at 6.65 average annual births per 1,000 people per year. The small population size of [Saint Pierre](https://en.wikipedia.org/wiki/Saint-Pierre,_Saint_Pierre_and_Miquelon) makes it easy to understand its presence on the list. With close margins, Japan (7.93), Andorra (8.13), and South Korea (8.19) occupy 3rd to 5th place, respectively.

This [Wikipedia article](https://en.wikipedia.org/wiki/Total_fertility_rate) provides some insights into the low birth rates recorded in Asian regions (Japan, South Korea, Taiwan, and Singapore). Aging and socio-economic challenges are described as the major reasons for these results.

### iii. Top 10 Countries with the Highest Death Rates

In [20]:
%%sql
SELECT name, death_rate
FROM facts
WHERE name <> 'World'
ORDER BY 2 DESC
LIMIT 10; 

 * sqlite:///factbook.db
Done.


name,death_rate
Lesotho,14.89
Ukraine,14.46
Bulgaria,14.44
Guinea-Bissau,14.33
Latvia,14.31
Chad,14.28
Lithuania,14.27
Namibia,13.91
Afghanistan,13.89
Central African Republic,13.8


In the 1st and 4th place are African countries Lesotho (14.89 deaths per 1,000 people) and Guinea Bissau (14.33), respectively. According to the [CDC](https://www.cdc.gov/globalhealth/countries/lesotho/default.htm) and [the Borgen project](https://borgenproject.org/top-diseases-in-guinea-bissau/#:~:text=The%20leading%20cause%20of%20death,significantly%20reduce%20their%20negative%20effects), the leading causes of death in these territories are infections and poverty.

The European countries Ukraine (14.46), Bulgaria (14.44), and Latvia (14.3) occupy the 2nd, 3rd and 5th place. Contrary to the African pattern, [studies](https://www.oecd-ilibrary.org/sites/82129230-en/1/3/2/1/4/index.html?itemId=/content/publication/82129230-en&_csp_=e7f5d56a7f4dd03271a59acda6e2be1b&itemIGO=oecd&itemContentType=book#:~:text=The%20main%20causes%20of%20death,or%2037%25%20of%20all%20deaths) reveal that the leading causes of death are non-infectious diseases like circulatory, digestive, or respiratory diseases and cancers. In Ukraine and Latvia, [poor healthcare administration](https://worldpopulationreview.com/country-rankings/death-rate-by-country) contributes to death.

As the handling of infectious diseases improves worldwide, sources have reported declining death rates in Lesotho and Guinea Bissau. At the same time, Latvia, Ukraine, and Bulgaria continue to experience increasing rates.

*Stats below taken from* [theGlobalEconomy.com](https://www.theglobaleconomy.com/)

**[Death Rates in Lesotho (1960 - 2020)](https://www.theglobaleconomy.com/Lesotho/Death_rate/)**

<img src="https://www.theglobaleconomy.com/graph_country.php?p=0&c=Lesotho&i=Death_rate" width=600 height=400/>

**[Death Rates in Guinea-Bissau (1960 - 2020)](https://www.theglobaleconomy.com/Guinea-Bissau/Death_rate/)**

<img src="https://www.theglobaleconomy.com/graph_country.php?p=0&c=Guinea-Bissau&i=Death_rate" width=600 height=400/>

**[Death Rates in Latvia (1960 - 2020)](https://www.theglobaleconomy.com/Latvia/Death_rate/)**

<img src="https://www.theglobaleconomy.com/graph_country.php?p=0&c=Latvia&i=Death_rate" width=600 height=400/>

**[Death Rates in Ukraine (1960 - 2020)](https://www.theglobaleconomy.com/Ukraine/Death_rate/)**

<img src="https://www.theglobaleconomy.com/graph_country.php?p=0&c=Ukraine&i=Death_rate" width=600 height=400/>

**[Death Rates in Bulgaria (1960 - 2020)](https://www.theglobaleconomy.com/Bulgaria/Death_rate/)**

<img src="https://www.theglobaleconomy.com/graph_country.php?p=0&c=Bulgaria&i=Death_rate" width=600 height=400/>

Unsurprisely, according to [ourworldindata.org](https://ourworldindata.org/grapher/birth-rate-vs-death-rate?tab=table&country=LSO~UKR~BGR~GNB~LVA) list of 2020, the top 10 countries by death rate did not include Lesotho and Guinea-Bissau. However, Bulgaria, Latvia, and Ukraine are present.

### iv. Top 10 Countries with the Lowest Death Rates

In [21]:
%%sql
SELECT name, death_rate
FROM facts
WHERE name <> 'World'
-- Filter out entries without death rates
    AND death_rate IS NOT NULL
ORDER BY 2
LIMIT 10; 

 * sqlite:///factbook.db
Done.


name,death_rate
Qatar,1.53
United Arab Emirates,1.97
Kuwait,2.18
Bahrain,2.69
Gaza Strip,3.04
Turks and Caicos Islands,3.1
Saudi Arabia,3.33
Oman,3.36
Singapore,3.43
West Bank,3.5


Qatar has the lowest mortality rate in the world at 1.53 deaths per 1,000 people. The UAE (1.97), Kuwait (2.8), and Bahrain (2.69) follow behind. These countries are renowned for their improved health care system, technologically advanced facilities, and ability to deliver some of the world's best patient care.

Here is another interesting pattern: Six of the ten countries on this list (Saudi Arabia, Kuwait, the United Arab Emirates, Qatar, Bahrain, and Oman) belong to the [Gulf Cooperation Council (GCC)](). In fact, all members of the council are present on our list above. The GCC is a political and economic alliance of the six aforementioned Middle Eastern countries.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1d/Gulf_Cooperation_Council.svg/250px-Gulf_Cooperation_Council.svg.png" width=400 height=400/>

*Source*: [Wikipedia](https://en.wikipedia.org/wiki/File:Gulf_Cooperation_Council.svg)

Arguably, the [most important](https://www.britannica.com/topic/Gulf-Cooperation-Council) goal of the GCC lies in Article 4 of its charter, which states that the alliance was formed to strengthen relations among its member countries and to promote cooperation among the countries' citizens. This alliance may hint at the positive effect of the countries on one another.

### v. Top 10 Countries with the Highest Birth-to-Death Ratio

Birth to death ratio can be calculated with the following formula:

$$
Birth\ to\ death\ ratio = \frac{birth\ rate}{death\ rate}
$$

In [22]:
%%sql
SElECT name, birth_rate, death_rate,
       ROUND((birth_rate / death_rate), 2) AS birth_to_death_ratio
FROM facts
WHERE name <> 'World'
ORDER BY 4 DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,birth_rate,death_rate,birth_to_death_ratio
Gaza Strip,31.11,3.04,10.23
Kuwait,19.91,2.18,9.13
Iraq,31.45,3.77,8.34
United Arab Emirates,15.43,1.97,7.83
Oman,24.44,3.36,7.27
Jordan,25.37,3.79,6.69
Solomon Islands,25.77,3.85,6.69
West Bank,22.99,3.5,6.57
Qatar,9.84,1.53,6.43
Vanuatu,25.04,4.09,6.12


The Gaza Strip records the highest birth-to-death ratio, with around 11 children born for every death in the territory. The rampant conflicts across the region make this a surprise.

The presence of Gulf Cooperation territories like Kuwait, UAE, Oman, and Qatar is expected, considering the low death rate in these regions.

### vi. Top 10 Countries with the Lowest Birth-to-Death Ratio

In [23]:
%%sql
SELECT name, birth_rate, death_rate,
       ROUND((birth_rate / death_rate), 2) AS birth_to_death_ratio
FROM facts
WHERE name <> 'World'
    AND birth_to_death_ratio NOT NULL
ORDER BY 4
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,birth_rate,death_rate,birth_to_death_ratio
Bulgaria,8.92,14.44,0.62
Serbia,9.08,13.66,0.66
Latvia,10.0,14.31,0.7
Lithuania,10.1,14.27,0.71
Hungary,9.16,12.73,0.72
Monaco,6.65,9.24,0.72
Germany,8.47,11.42,0.74
Slovenia,8.42,11.37,0.74
Ukraine,10.72,14.46,0.74
Saint Pierre and Miquelon,7.42,9.72,0.76


All countries and territories on the list above are located in Europe. The compounded effect of decreasing birth rates and increasing deaths may pose a potential risk. The ratio indicates a struggle in Europe to achieve a single birth for every ten deaths. An article emphasizing a similar concern can be found [here](https://www.theguardian.com/world/2015/aug/23/baby-crisis-europe-brink-depopulation-disaster).

The struggle to maintain corresponding birth rates for every death in Europe might be made clearer when compared to other continents. [OurWorldinData](https://ourworldindata.org/grapher/birth-rate-vs-death-rate) conducted a similar analysis of birth and death rates across the world's countries in 2021; the results are enlightening.

<img src="https://ourworldindata.org/grapher/exports/birth-rate-vs-death-rate.svg" width=800 height=300/>

We can tell from the chart above; when compared to other continents, Europe recorded higher than average death rates and lower than average birth rates. An inherent message here is that several countries appear to be shrinking from a population perspective. We can identify countries with the fastest declining populations to shred some light on this alarming discovery.

## Countries with the Fastest Declining Population

To identify the fastest declining populations, we select countries that meed the following criteria:

- A birth-to-death ratio of less than one (indicates the absence of natural increase)
- Little to no population growth (population growth of less than 1%)
- Very low immigration levels (migration rate of less than one)

In [24]:
%%sql
SELECT name,
        ROUND((birth_rate / death_rate), 2) AS birth_death_ratio,
        population_growth,
        migration_rate
FROM facts
WHERE birth_death_ratio < 1
    AND population_growth < 1
    AND migration_rate < 1
ORDER BY 4, 2, 3;

 * sqlite:///factbook.db
Done.


name,birth_death_ratio,population_growth,migration_rate
Serbia,0.66,0.46,0.0
Japan,0.83,0.16,0.0
Romania,0.77,0.3,0.24
Bulgaria,0.62,0.58,0.29
Slovenia,0.74,0.26,0.37
Bosnia and Herzegovina,0.91,0.13,0.38
Poland,0.96,0.09,0.46
Belarus,0.8,0.2,0.7


Asides from Japan, all the countries in the above list are located in or near Eastern Europe.

<img src="https://www.researchgate.net/profile/Boguslawa-Dobek-Ostrowska-2/publication/286923130/figure/fig1/AS:669496704516105@1536631861379/Map-of-Central-and-Eastern-Europe-CEE.ppm" width=350 height=400/>

Source: [ResearchGate](https://www.researchgate.net/figure/Map-of-Central-and-Eastern-Europe-CEE_fig1_286923130)

The first cause of the rapidly declining population around East Europe is birth rates. According to the Peterson Institute for International Economics (PIIE), the region's birth rates have fallen since [the collapse of the Soviet Union](https://www.visualcapitalist.com/4-historical-maps-that-explain-the-ussr/).

The falling birth rate is also not enough to offset deaths and citizens leaving their country to live elsewhere. Eastern Europe saw several waves of emigration following the European Union's (EU) border expansions in 2004 and 2007. [The PIIE reports](https://www.visualcapitalist.com/ranked-the-20-countries-with-the-fastest-declining-populations/) that by 2016, 6.3 million Eastern Europeans had emigrated to other EU countries.

As far as Japan is concerned, birth rates have fallen continuously since 1970. Japan's fertility rate is plotted against various forecasts over the years. These forecasts headed upwards, but in reality [rates decreased consistently everytime](https://flowingdata.com/2015/01/08/japan-fertility-rate-forecasts-versus-reality/):

<img src="https://flowingdata.com/wp-content/uploads/2015/01/Japan-fertility-forecasts-620x424.png" width=650 height=400/>

It wasn't until 2010, however, that the country's overall population began to shrink. The situation appears dire. In 2021, 811,604 babies were born in Japan, while 1.44 million people died. The Japanese government has introduced various social programs to make having kids more appealing. Still, these don't appear to be getting to the root of the problem. For deeper insight into Japan's low birth rates, it's worth reading [this article](https://www.theatlantic.com/business/archive/2017/07/japan-mystery-low-birth-rate/534291/) by *The Atlantic*.

## Conclusion

Throughout this project, we used the CIA World factbook data to identify the leading and lagging territories of the world. Our analysis is based on demographic indicators like population, mortality, and fertility.

> - Accounting for 46% of the world's total population, China, India, the United States, Indonesia, and Brazil are the most populated countries. The least populated world locations are majorly Islands and dependencies.
-Due to their small land area, Macau, Monaco, Singapore, and Hong Kong are more densely populated than the rest of the world. However, when considering larger land areas, Bangladesh, South Korea, and Japan are the most densely populated countries.
-Fertility is highest in Africa and lowest in Europe and Asia. Some African countries like Lesotho, Guinea-Bissau and Eastern European countries like Bulgaria, Lithuania, and Serbia record the highest death rates. The Gulf cooperation countries of the Middle-East record the lowest death rates.
-Population is rapidly declining in East and South-East Europe. This occurrence is due to reducing birth rates, increasing deaths, and emigration to other countries. Japan is also experiencing a population decline despite the positive forecasts for the country.

## Limitation

It appears that the CIA collated this data between 2014 and 2015. As a result, our findings are limited to the period the data was published. They may not reflect the current state of the World's countries.